In [ ]:
import numpy as np
import cvxopt.base
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from ctypes import *

%run "~\Documents\GitHub\ProjetAnnuel3IBD\projet\python\encapsulate\encapsulateSharedMethods.py"
pathDLL = "C:/Users/nico_/Documents/GitHub/ProjetAnnuel3IBD/projet/MLAlgorithms/ML_Library/x64/Release/ML_Library.dll"
myDll = CDLL(pathDLL)

In [ ]:


def getAlpha(bigMatrix, Y):
# Define problem data
    q = np.ones(bigMatrix.shape[0])
    q = cvxopt.matrix(-q)

    n = bigMatrix.shape[0]

    bigMatrix = cvxopt.matrix(bigMatrix)
    A = cvxopt.matrix(Y.T, (1, n))

    b = cvxopt.matrix(0.0)
    G = cvxopt.matrix(np.diag(np.ones(n) * -1.0))
    h = cvxopt.matrix(np.zeros(n))
    cvxopt.solvers.options['show_progress'] = False
    solution = cvxopt.solvers.qp(bigMatrix, q, G, h, A, b)
    alphas = np.ravel(solution['x'])
    return alphas

def getBigMatrix(X, Y):
    bigMatrix = np.empty(shape=(X.shape[0], X.shape[0]))
    for i in range(0, X.shape[0]):
        for j in range(0, X.shape[0]):
            bigMatrix[i][j] = np.dot(X[i],np.transpose(X[j])) * (Y[i] * Y[j])
    return bigMatrix



In [ ]:
Y = [ 1, 1, -1, -1, -1]
Xnp = np.array([ [1, 1], [2, 1], [2, 2], [4, 1], [4, 4]], dtype=float)
X = Xnp.flatten().tolist()
Ynp = np.array(Y, dtype=float)

bigMatrix = getBigMatrix(Xnp, Ynp)
alpha = getAlpha(bigMatrix, Ynp)

# Load Matrix X
pMatrixX = loadTestCase(myDll, X, Xnp.shape[0], Xnp.shape[1], 0)

# Load Matrix Y
pMatrixY = loadTestCase(myDll, Y, Ynp.shape[0], 1, 0)

alphaVector = datasetToVector(myDll,alpha, 0)
myDll.fitSvm.argtypes = [c_void_p, c_void_p, c_void_p]
myDll.fitSvm(pMatrixX, pMatrixY, alphaVector)